In [2]:
import csv
import requests
from bs4 import BeautifulSoup

# Function to extract links from a given URL
def extract_links(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = [a['href'] for a in soup.find_all('a', href=True)]
        return links
    except Exception as e:
        print(f"Error extracting links from {url}: {str(e)}")
        return []

# Initialize an empty dictionary to store the data
link_data = {}

# Read the CSV file containing links
csv_filename = 'outfile.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for row in reader:
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        
        # Assuming the link is in the second column and the index is in the first column
        index = row[0]
        link = row[1]
        
        # Extract links from the current URL and store them as values for the key (current link)
        link_data[link] = extract_links(link)

# Print the resulting dictionary
for key, values in link_data.items():
    print(f"Link: {key}")
    print("Links found on the page:")
    for value in values:
        print(f" - {value}")
    print()

# You can also save this data to a new CSV file if needed
# Example:
# with open('link_data.csv', 'w', newline='') as output_csv:
#     writer = csv.writer(output_csv)
#     for key, values in link_data.items():
#         writer.writerow([key] + values)


Error extracting links from : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Error extracting links from http://www.risepizzeria.com/: HTTPConnectionPool(host='www.risepizzeria.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb3f23e4ac0>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
Error extracting links from http://www.charliepalmersteak.com/locations/napa: HTTPConnectionPool(host='www.charliepalmersteak.com', port=80): Max retries exceeded with url: /locations/napa (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb3f208c490>: Failed to establish a new connection: [Errno -5] No address associated with hostname'))
Error extracting links from http://www.lacabana.vista.com/: HTTPConnectionPool(host='www.lacabana.vista.com', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HT

KeyboardInterrupt: 

In [12]:
import csv
import requests
from bs4 import BeautifulSoup
import re  
import time  
from requests.exceptions import Timeout  # Import the Timeout exception for links which takes too much time to load

# Function to categorize links on a webpage
def categorize_links(links):
    maps_link = ""
    instagram_link = ""
    facebook_link = ""
    twitter_link = ""
    other_links = []

    for link in links:
        if "maps.google.com" in link:
            maps_link = link
        elif "www.instagram.com" in link:
            instagram_link = link
        elif "www.facebook.com" in link:
            facebook_link = link
        elif "twitter.com" in link:
            twitter_link = link
        else:
            other_links.append(link)

    return [maps_link, instagram_link, facebook_link, twitter_link] + other_links


def extract_links_with_error_handling(url, index):
    retries = 3  # Number of retries before giving up
    retry_delay = 2  # Delay between retries in seconds
    for _ in range(retries):
        try:
            response = requests.get(url, timeout=10)  # Set a timeout of 10 seconds
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]
            return list(set(valid_links))  # Remove duplicates by converting to a set and back to a list
        except Timeout:
            print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
            time.sleep(retry_delay)
        except ConnectionError:
            print(f"Connection error occurred for index {index} and URL {url}. Retrying...")
            time.sleep(retry_delay)
        except Exception as e:
            print(f"Error extracting links from index {index} for URL {url}: {str(e)}")
            return []

# Initialize an empty dictionary to store the data
link_data = {}

# Read the CSV file containing links
csv_filename = 'outfile_small.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for index, row in enumerate(reader):
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        
        # Assuming the link is in the second column and the index is in the first column
        link_index = row[0]
        link = row[1]
        if(link == ''):
            continue
        print('going for ', link_index, ' : ', link)
        # Extract links from the current URL and store them as values for the key (current link)
        link_data[link] = extract_links(link, link_index)

        # Introduce a delay of 2 seconds before processing the next link
        time.sleep(1)  # You can adjust the sleep duration as needed

# Categorize links after extracting all links
for key, values in link_data.items():
    link_data[key] = categorize_links(values)

# Print the resulting dictionary and save it to an output file
output_filename = 'link_data.csv'
with open(output_filename, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    for key, values in link_data.items():
        print(f"Link: {key}")
        print("Links found on the page:")
        writer.writerow([key] + values)
        for value in values:
            print(f" - {value}")
        print()

print(f"Data saved to {output_filename}")


going for  0  :  http://www.yannistaverna.com/
going for  1  :  http://www.starbellysf.com/
going for  2  :  http://www.manzonirestaurant.com/
going for  3  :  http://www.macaronigrill.com/
going for  4  :  https://www.pfchangs.com/locations/us/ca/pleasanton/1330-stoneridge-mall-rd/9819-pleasanton.html
going for  5  :  https://www.pfchangs.com/locations/us/ca/sunnyvale/390-w-el-camino-real/6900-sunnyvale.html
going for  6  :  https://www.pfchangs.com/locations/us/ca/palo-alto/900-stanford-shopping-center/9911-palo-alto.html
going for  7  :  http://www.kenzonapa.com/
going for  9  :  http://monsoonhimalayancuisine.com/
Error extracting links from index 9 for URL http://monsoonhimalayancuisine.com/: 403 Client Error: Forbidden for url: https://monsoonhimalayancuisine.com/
going for  10  :  http://www.risepizzeria.com/
Link: http://www.yannistaverna.com/
Links found on the page:
 - 
 - 
 - 
 - 
 - https://cutt.ly/daftarmaha4d
 - https://secure.livechatinc.com/licence/12759861/v2/open_chat

In [24]:
import csv
import requests
from bs4 import BeautifulSoup
import re  
import time  
from requests.exceptions import Timeout, RequestException  # Import the Timeout exception for links which takes too much time to load

# Function to categorize links on a webpage
def categorize_links(links):
    if links is None:
        return [] 
    maps_link = ""
    instagram_link = ""
    facebook_link = ""
    twitter_link = ""
    other_links = []

    for link in links:
        if "maps.google.com" in link:
            maps_link = link
        elif "www.instagram.com" in link:
            instagram_link = link
        elif "www.facebook.com" in link:
            facebook_link = link
        elif "twitter.com" in link:
            twitter_link = link
        else:
            other_links.append(link)

    return [maps_link, instagram_link, facebook_link, twitter_link] + other_links


def extract_links_with_error_handling(url, index):
    retries = 3  # Number of retries before giving up
    retry_delay = 2  # Delay between retries in seconds
    for _ in range(retries):
        try:
            print('heyaaaa')
            response = requests.get(url, timeout=(5, 5))  # Set a timeout of 10 seconds for both connect and read
            print('aaaayeh')
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]
            return list(set(valid_links))  # Remove duplicates by converting to a set and back to a list
        except Timeout:
            print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
            time.sleep(retry_delay)
        except RequestException as e:
            print(f"RequestException occurred for index {index} and URL {url}: {str(e)}")
            return []  # Exit the loop and move to the next link
        except Exception as e:
            print(f"Error extracting links from index {index} for URL {url}: {str(e)}")
            return []

# Initialize an empty dictionary to store the data
link_data = {}

# Read the CSV file containing links
csv_filename = 'outfile.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for index, row in enumerate(reader):
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        
        # Assuming the link is in the second column and the index is in the first column
        link_index = row[0]
        link = row[1]
        if(link == ''):
            continue
        print('going for ', link_index, ' : ', link)
        # Extract links from the current URL and store them as values for the key (current link)
        link_data[link] = extract_links_with_error_handling(link, link_index)

        # Introduce a delay of 2 seconds before processing the next link
        time.sleep(1)  # You can adjust the sleep duration as needed

# Categorize links after extracting all links
for key, values in link_data.items():
    link_data[key] = categorize_links(values)

# Print the resulting dictionary and save it to an output file
output_filename = 'link_data_full.csv'
with open(output_filename, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    for key, values in link_data.items():
        print(f"Link: {key}")
        print("Links found on the page:")
        writer.writerow([key] + values)
        for value in values:
            print(f" - {value}")
        print()

print(f"Data saved to {output_filename}")


going for  0  :  http://www.yannistaverna.com/
heyaaaa
aaaayeh
going for  1  :  http://www.starbellysf.com/
heyaaaa
aaaayeh
going for  2  :  http://www.manzonirestaurant.com/
heyaaaa
aaaayeh
going for  3  :  http://www.macaronigrill.com/
heyaaaa
aaaayeh
going for  4  :  https://www.pfchangs.com/locations/us/ca/pleasanton/1330-stoneridge-mall-rd/9819-pleasanton.html
heyaaaa
aaaayeh
going for  5  :  https://www.pfchangs.com/locations/us/ca/sunnyvale/390-w-el-camino-real/6900-sunnyvale.html
heyaaaa
aaaayeh
going for  6  :  https://www.pfchangs.com/locations/us/ca/palo-alto/900-stanford-shopping-center/9911-palo-alto.html
heyaaaa
aaaayeh
going for  7  :  http://www.kenzonapa.com/
heyaaaa
aaaayeh
going for  9  :  http://monsoonhimalayancuisine.com/
heyaaaa
aaaayeh
RequestException occurred for index 9 and URL http://monsoonhimalayancuisine.com/: 403 Client Error: Forbidden for url: https://monsoonhimalayancuisine.com/
going for  10  :  http://www.risepizzeria.com/
heyaaaa
aaaayeh
going for 

In [6]:
import csv
import requests
from bs4 import BeautifulSoup
import re  
import time  
from requests.exceptions import Timeout, RequestException  # Import the Timeout exception for links which takes too much time to load

# Function to categorize links on a webpage
def categorize_links(links):
    if links is None:
        return [] 
    maps_link = ""
    instagram_link = ""
    facebook_link = ""
    twitter_link = ""
    other_links = []

    for link in links:
        if "maps.google.com" in link:
            maps_link = link
        elif "www.instagram.com" in link:
            instagram_link = link
        elif "www.facebook.com" in link:
            facebook_link = link
        elif "twitter.com" in link:
            twitter_link = link
        else:
            other_links.append(link)

    return [maps_link, instagram_link, facebook_link, twitter_link] + other_links


def extract_links_with_error_handling(url, index, visited_links=None, depth=0, max_depth=2):
    if visited_links is None:
        visited_links = set()

    retries = 3  # Number of retries before giving up
    retry_delay = 2  # Delay between retries in seconds
    unique_links = set()

    try:
        if url not in visited_links and depth <= max_depth:
            visited_links.add(url)
            print(f'Crawling {url}')
            response = requests.get(url, timeout=(5, 5))  # Set a timeout of 10 seconds for both connect and read
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]

            for link in valid_links:
                if not any(social in link for social in ["whatsapp", "insta", "facebook", "twitter"]):
                    unique_links.add(link)

            for link in unique_links.copy():
                unique_links |= extract_links_with_error_handling(link, index, visited_links, depth + 1, max_depth)

    except Timeout:
        print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
        time.sleep(retry_delay)
    except RequestException as e:
        print(f"RequestException occurred for index {index} and URL {url}: {str(e)}")
    except Exception as e:
        print(f"Error extracting links from index {index} for URL {url}: {str(e)}")

    return unique_links


# Initialize an empty dictionary to store the data
link_data = {}

# Read the CSV file containing links
csv_filename = 'outfile_small.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for index, row in enumerate(reader):
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        link_index = row[0]
        link = row[1]
        if(link == ''):
            continue
        print(' ---------------------------going for-----------------------------\n ', link_index, ' : ', link)
        visited_links = set()  
        max_depth = 1
        link_data[link] = extract_links_with_error_handling(link, link_index, visited_links, depth=0, max_depth=max_depth)

        time.sleep(1)  # You can adjust the sleep duration as needed

# Categorize links after extracting all links
for key, values in link_data.items():
    link_data[key] = categorize_links(values)

# Print the resulting dictionary and save it to an output file
output_filename = 'deepcrawl/links2.csv'
with open(output_filename, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    for key, values in link_data.items():
        print(f"Link: {key}")
        print("Links found on the page:")
        writer.writerow([key] + values)
        for value in values:
            print(f" - {value}")
        print()

print(f"Data saved to {output_filename}")


 ---------------------------going for-----------------------------
  0  :  http://www.yannistaverna.com/
Crawling http://www.yannistaverna.com/
Crawling https://secure.livechatinc.com/licence/12759861/v2/open_chat.cgi?
Crawling https://www.yannistaverna.com/
Crawling https://cutt.ly/daftarmaha4d
Crawling https://cutt.ly/maha4d
 ---------------------------going for-----------------------------
  1  :  http://www.starbellysf.com/
Crawling http://www.starbellysf.com/
Crawling https://starbellysf.cardfoundry.com/giftcards.php


KeyboardInterrupt: 

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import re  
import time  
from requests.exceptions import Timeout, RequestException  # Import the Timeout exception for links which takes too much time to load

# Function to categorize links on a webpage
def categorize_links(links):
    if links is None:
        return [] 
    maps_link = ""
    instagram_link = ""
    facebook_link = ""
    twitter_link = ""
    other_links = []

    for link in links:
        if "maps.google.com" in link:
            maps_link = link
        elif "www.instagram.com" in link:
            instagram_link = link
        elif "www.facebook.com" in link:
            facebook_link = link
        elif "twitter.com" in link:
            twitter_link = link
        else:
            other_links.append(link)

    return [maps_link, instagram_link, facebook_link, twitter_link] + other_links


def extract_links_with_error_handling(url, index, visited_links=None, depth=0, max_depth=2):
    if visited_links is None:
        visited_links = set()

    retries = 3  # Number of retries before giving up
    retry_delay = 2  # Delay between retries in seconds
    unique_links = set()

    try:
        if url not in visited_links and depth <= max_depth:
            visited_links.add(url)
            print(f'Crawling {url}')
            response = requests.get(url, timeout=(5, 5))  # Set a timeout of 10 seconds for both connect and read
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]

            for link in valid_links:
                if not any(social in link for social in ["whatsapp", "insta", "facebook", "twitter"]):
                    unique_links.add(link)

            for link in unique_links.copy():
                unique_links |= extract_links_with_error_handling(link, index, visited_links, depth + 1, max_depth)

    except Timeout:
        print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
        time.sleep(retry_delay)
    except RequestException as e:
        print(f"RequestException occurred for index {index} and URL {url}: {str(e)}")
    except Exception as e:
        print(f"Error extracting links from index {index} for URL {url}: {str(e)}")

    return unique_links


# Initialize an empty dictionary to store the data
link_data = {}

# Read the CSV file containing links
csv_filename = 'outfile.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for index, row in enumerate(reader):
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        link_index = row[0]
        link = row[1]
        if(link == ''):
            continue
        print(' ---------------------------going for-----------------------------\n ', link_index, ' : ', link)
        visited_links = set()  
        max_depth = 1
        link_data[link] = extract_links_with_error_handling(link, link_index, visited_links, depth=0, max_depth=max_depth)

        time.sleep(1) 

# Categorize links after extracting all links
for key, values in link_data.items():
    link_data[key] = categorize_links(values)

# Print the resulting dictionary and save it to an output file
output_filename = 'deepcrawl/links_large.csv'
with open(output_filename, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    for key, values in link_data.items():
        print(f"Link: {key}")
        print("Links found on the page:")
        writer.writerow([key] + values)
        for value in values:
            print(f" - {value}")
        print()

print(f"Data saved to {output_filename}")


threading

In [8]:
import csv
import requests
from bs4 import BeautifulSoup
import re  
import time  
import threading
from requests.exceptions import Timeout, RequestException  # Import the Timeout exception for links which takes too much time to load

# Define a lock to synchronize access to link_data
link_data_lock = threading.Lock()

# Function to crawl a single link and store the result in link_data
def crawl_link(link_index, link, max_depth):
    visited_links = set()
    result = extract_links_with_error_handling(link, link_index, visited_links, depth=0, max_depth=max_depth)
    
    with link_data_lock:
        link_data[link] = result

# Function to categorize links on a webpage
def categorize_links(links):
    if links is None:
        return [] 
    maps_link = ""
    instagram_link = ""
    facebook_link = ""
    twitter_link = ""
    other_links = []

    for link in links:
        if "maps.google.com" in link:
            maps_link = link
        elif "www.instagram.com" in link:
            instagram_link = link
        elif "www.facebook.com" in link:
            facebook_link = link
        elif "twitter.com" in link:
            twitter_link = link
        else:
            other_links.append(link)

    return [maps_link, instagram_link, facebook_link, twitter_link] + other_links


def extract_links_with_error_handling(url, index, visited_links=None, depth=0, max_depth=2):
    if visited_links is None:
        visited_links = set()

    retries = 3  # Number of retries before giving up
    retry_delay = 2  # Delay between retries in seconds
    unique_links = set()

    try:
        if url not in visited_links and depth <= max_depth:
            visited_links.add(url)
            print(f'Crawling {url}')
            response = requests.get(url, timeout=(5, 5))  # Set a timeout of 10 seconds for both connect and read
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            links = [a['href'] for a in soup.find_all('a', href=True)]
            valid_links = [link for link in links if re.match(r'^https?://', link)]

            for link in valid_links:
                if not any(social in link for social in ["whatsapp", "insta", "facebook", "twitter"]):
                    unique_links.add(link)

            for link in unique_links.copy():
                unique_links |= extract_links_with_error_handling(link, index, visited_links, depth + 1, max_depth)

    except Timeout:
        print(f"Timeout occurred for index {index} and URL {url}. Retrying...")
        time.sleep(retry_delay)
    except RequestException as e:
        print(f"RequestException occurred for index {index} and URL {url}: {str(e)}")
    except Exception as e:
        print(f"Error extracting links from index {index} for URL {url}: {str(e)}")

    return unique_links


# Initialize an empty dictionary to store the data
link_data = {}
threads = []
# Read the CSV file containing links
csv_filename = 'outfile.csv'  # Change this to your CSV file name
with open(csv_filename, 'r') as csvfile:
    reader = csv.reader(csvfile)
    
    # Iterate over each row in the CSV file
    for index, row in enumerate(reader):
        if len(row) < 2:
            continue  # Skip rows with less than 2 columns
        link_index = row[0]
        link = row[1]
        if(link == ''):
            continue
        print(' ---------------------------going for-----------------------------\n ', link_index, ' : ', link)
        visited_links = set()  
        max_depth = 1
        
		    # Create and start a new thread for each link
        thread = threading.Thread(target=crawl_link, args=(link_index, link, max_depth))
        thread.start()
        threads.append(thread)
        # time.sleep(1) 

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Categorize links after extracting all links
for key, values in link_data.items():
    link_data[key] = categorize_links(values)

# Print the resulting dictionary and save it to an output file
output_filename = 'deepcrawl/links_large.csv'
with open(output_filename, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    for key, values in link_data.items():
        print(f"Link: {key}")
        print("Links found on the page:")
        writer.writerow([key] + values)
        for value in values:
            print(f" - {value}")
        print()

print(f"Data saved to {output_filename}")


 ---------------------------going for-----------------------------
  0  :  http://www.yannistaverna.com/
Crawling http://www.yannistaverna.com/
 ---------------------------going for-----------------------------
  1  :  http://www.starbellysf.com/
Crawling http://www.starbellysf.com/
 ---------------------------going for-----------------------------
  2  :  http://www.manzonirestaurant.com/
Crawling http://www.manzonirestaurant.com/
 ---------------------------going for-----------------------------
  3  :  http://www.macaronigrill.com/
Crawling http://www.macaronigrill.com/
 ---------------------------going for-----------------------------
  4  :  https://www.pfchangs.com/locations/us/ca/pleasanton/1330-stoneridge-mall-rd/9819-pleasanton.html
Crawling https://www.pfchangs.com/locations/us/ca/pleasanton/1330-stoneridge-mall-rd/9819-pleasanton.html
 ---------------------------going for-----------------------------
  5  :  https://www.pfchangs.com/locations/us/ca/sunnyvale/390-w-el-camino-